In [2]:
import numpy as np
import sympy as sp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter

To compute geodesic on a surface 

1. Compute Christofell Symbols
$$\Gamma_{ij}^{k} = \frac{\partial^2\vec{R}}{\partial u^i \partial u^j}\cdot\frac{\partial \vec{R}}{\partial u^{l}} \large\widetilde{g}^{lk}$$

2. Solve Geodesic Equations
$$\frac{d^2u^k}{d\lambda^2} + \Gamma_{ij}^{k}\frac{du^i}{d\lambda}\frac{du^j}{d\lambda} = 0$$

In [3]:
u1,u2,X,Y,Z=sp.symbols('u1 u2 X Y Z')
R=sp.symbols('R',cls=sp.Function)

u=sp.Matrix([u1,u2])

# θ=θ(t)
# φ=φ(t)
# θ_d=sp.diff(θ,φ)
# # θ_d

In [4]:
#assigning values to X,Y,Z
X1=sp.cos(u[1])*sp.sin(u[0])
Y1=sp.sin(u[1])*sp.sin(u[0])
Z1=sp.cos(u[0])
R=X1+Y1+Z1
R

sin(u1)*sin(u2) + sin(u1)*cos(u2) + cos(u1)

In [5]:
xx=[X1,Y1,Z1]
d=2

In [6]:
def dot_p(A,B,d):
    sum=0
    for i in range(d):
        sum+=A[i]*B[i]
    return sum

In [7]:
def dv(u,xx,d):
    sum=[]
    for i in range(d+1):
        sum.append(sp.diff(xx[i],u))
    return sum

In [8]:
Rdv=[]
for i in range(d):
    Rdv.append(dv(u[i],xx,d))
Rdv

[[cos(u1)*cos(u2), sin(u2)*cos(u1), -sin(u1)],
 [-sin(u1)*sin(u2), sin(u1)*cos(u2), 0]]

In [9]:
R_2dv=[]
for i in range(d):
    t=[]
    for j in range(d):
        t.append(dv(u[j],Rdv[i],d))
    R_2dv.append(t)
R_2dv

[[[-sin(u1)*cos(u2), -sin(u1)*sin(u2), -cos(u1)],
  [-sin(u2)*cos(u1), cos(u1)*cos(u2), 0]],
 [[-sin(u2)*cos(u1), cos(u1)*cos(u2), 0],
  [-sin(u1)*cos(u2), -sin(u1)*sin(u2), 0]]]

In [10]:
M_T=[]  
for i in range(d):
    t=[]
    for j in range(d):
        t.append(sp.trigsimp(dot_p(Rdv[i],Rdv[j],d+1)))
    M_T.append(t)

       
M_T

[[1, 0], [0, sin(u1)**2]]

In [11]:
M_Tm=sp.Matrix([[M_T[0][0],M_T[0][1]],[M_T[1][0],M_T[1][1]]])
M_Tinv=M_Tm.inv()
M_Tinv
                                       

Matrix([
[1,             0],
[0, sin(u1)**(-2)]])

In [12]:
M_inv=[]
tt=0
for i in range(d):
    t=[]
    for j in range(d):
        t.append(M_Tinv[tt])
        tt+=1
    M_inv.append(t)
M_inv

[[1, 0], [0, sin(u1)**(-2)]]

$\Lambda--->\large{\frac{\partial^2\vec{R}}{\partial u^i \partial u^j}\cdot\frac{\partial \vec{R}}{\partial u^{l}}}$

In [13]:
#
Λ=[]
for x in range(d):
    t1=[]
    for i in range(d):
        t2=[]
        for j in range(d):
            t2.append(sp.simplify(dot_p(R_2dv[i][j],Rdv[x],d+1)))
        t1.append(t2)
    Λ.append(t1)
Λ

[[[0, 0], [0, -sin(2*u1)/2]], [[0, sin(2*u1)/2], [sin(2*u1)/2, 0]]]

$\chi ---> \Lambda\large\widetilde{g}^{lk}$

In [14]:
χ=[]
# inverse metrictensor multipication
for y in range(d):
    tc=[]
    for x in range(d):
        t1=[]
        for i in range(d):
            t2=[]
            for j in range(d):
                t2.append(Λ[x][i][j]*M_inv[x][y])
            t1.append(t2)
        tc.append(t1)
    χ.append(tc)
χ 
            
    

[[[[0, 0], [0, -sin(2*u1)/2]], [[0, 0], [0, 0]]],
 [[[0, 0], [0, 0]],
  [[0, sin(2*u1)/(2*sin(u1)**2)], [sin(2*u1)/(2*sin(u1)**2), 0]]]]

In [15]:
χ[0]

[[[0, 0], [0, -sin(2*u1)/2]], [[0, 0], [0, 0]]]

$\Gamma_1 ---> \Gamma^{1}_{ij}$

In [16]:
t1=[]
for i in range(d):
    t=[]
    for j in range(d):
        t.append(χ[0][0][i][j]+χ[0][1][i][j])
    t1.append(t)
t1
    

[[0, 0], [0, -sin(2*u1)/2]]

$\Gamma1--->\Gamma^1_{ij}$ as  [i,j,value]

In [17]:
Γ1=[]
for i in range(d):
    t=[]
    for j in range(d):
        if t1[i][j]!=0:
            t.append(i+1)
            t.append(j+1)
            t.append(t1[i][j])
            Γ1.append(t)
Γ1

[[2, 2, -sin(2*u1)/2]]

In [18]:
t2=[]
for i in range(d):
    t=[]
    for j in range(d):
        t.append(χ[1][0][i][j]+χ[1][1][i][j])
    t2.append(t)
t2
    

[[0, sin(2*u1)/(2*sin(u1)**2)], [sin(2*u1)/(2*sin(u1)**2), 0]]

$\Gamma_2 ---> \Gamma^{2}_{ij}$ as [i,j,value]

In [19]:
Γ2=[]
for i in range(d):
    t=[]
    for j in range(d):
        if t2[i][j]!=0 and i!=j:
            t.append(i+1)
            t.append(j+1)
            t.append(t2[i][j])
            Γ2.append(t)
Γ2

[[1, 2, sin(2*u1)/(2*sin(u1)**2)], [2, 1, sin(2*u1)/(2*sin(u1)**2)]]

Consider only diagonal elements and one of symmeteric elements of Chritofell symbol, $\Gamma$.

$\Gamma ---> [\Gamma^{1}_{ij},\Gamma^{2}_{ij}]$

In [20]:
Γ=[Γ1[0],Γ2[0]]
Γ

[[2, 2, -sin(2*u1)/2], [1, 2, sin(2*u1)/(2*sin(u1)**2)]]

Simplified values:

In [21]:
for i in range(len(Γ)):
    Γ[i][2]=sp.trigsimp(Γ[i][2])
Γ

[[2, 2, -sin(2*u1)/2], [1, 2, 1/tan(u1)]]

$\Gamma[0]$ = $\Gamma^{1}_{22}$ and $\Gamma[1]$ = $\Gamma^{2}_{12}$